#### Melakukan Esktraksi Ciri Citra

In [15]:
# Package yang dibutuhkan

import numpy as np
import pandas as pd
from skimage.io import imread, imshow
from skimage.color import rgb2hsv
from skimage.morphology import binary_erosion, binary_dilation

In [16]:
# Dataframe untuk menampung hasil ekstraksi ciri

df = pd.DataFrame({'mature': [], 'immature': [], 'rotten': [], 'label': []})

In [17]:
# Fungsi untuk pemeriksaan key pada ekstraksi ciri
def check_key(dic):
    if len(dic.keys()) == 1:
        if list(dic.keys())[0] == True:
            dic[False] = 0
        else:
            dic[True] = 0
        return dic
    return dic

In [18]:
# Melakukan ekstraksi ciri secara otomatis melalui perulangan

labels = ['mature', 'immature', 'rotten']

for label in labels:

    for i in range(7, 10):
        
        print(f'picture/{label}/{label} ({i + 1}).jpg')
        
        # Mengambil gambar dari setiap folder
        img = imread(f'picture/{label}/{label} ({i + 1}).jpg')
        imgHSV = rgb2hsv(img)
        
        # Membuat mask untuk segmentasi

        # Mask seluruh buah
        lower_mask = imgHSV[:,:,0] <= 1
        upper_mask = imgHSV[:,:,0] >= 0
        saturation_mask = imgHSV[:,:,1] > 0.25
        all_mask = upper_mask * lower_mask * saturation_mask

        # Melakukan operasi morfologi pada mask buah
        morf_mask = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])
        all_mask = binary_dilation(all_mask, morf_mask)
        all_mask = binary_erosion(all_mask, morf_mask)

        # Mask matang
        # 1
        lower_mask = imgHSV[:,:,0] >= 0
        upper_mask = imgHSV[:,:,0] < 0.045
        saturation_mask = imgHSV[:,:,1] > 0.25
        value_mask = imgHSV[:,:,2] <= 1
        mature_mask_lower = lower_mask * upper_mask * saturation_mask * value_mask
        # 2
        lower_mask = imgHSV[:,:,0] > 0.9
        upper_mask = imgHSV[:,:,0] <= 1
        saturation_mask = imgHSV[:,:,1] > 0.25
        mature_mask_upper = lower_mask * upper_mask * saturation_mask
        # Final mask
        mature_mask = mature_mask_lower + mature_mask_upper

        # Mask belum matang
        lower_mask = imgHSV[:,:,0] > 0.175
        upper_mask = imgHSV[:,:,0] < 0.5
        saturation_mask = imgHSV[:,:,1] > 0.5
        immature_mask = lower_mask * upper_mask * saturation_mask

        # Mask busuk
        if label in ['mature', 'rotten']:
            rotten_mask = all_mask ^ mature_mask
        else:
            rotten_mask = all_mask ^ immature_mask

        # Ekstraksi ciri
        unique, counts = np.unique(all_mask, return_counts=True)
        allMaskCount = check_key(dict(zip(unique, counts)))[True]

        unique, counts = np.unique(mature_mask, return_counts=True)
        matureMaskCount = check_key(dict(zip(unique, counts)))[True]

        unique, counts = np.unique(rotten_mask, return_counts=True)
        rottenMaskCount = check_key(dict(zip(unique, counts)))[True]

        unique, counts = np.unique(immature_mask, return_counts=True)
        immatureMaskCount = check_key(dict(zip(unique, counts)))[True]

        # Tambahkan hasil ekstraksi data ke dalam dataframe
        df.loc[len(df.index)] = [matureMaskCount/allMaskCount, immatureMaskCount/allMaskCount, rottenMaskCount/allMaskCount, label]

picture/mature/mature (8).jpg
picture/mature/mature (9).jpg
picture/mature/mature (10).jpg
picture/immature/immature (8).jpg
picture/immature/immature (9).jpg
picture/immature/immature (10).jpg
picture/rotten/rotten (8).jpg
picture/rotten/rotten (9).jpg
picture/rotten/rotten (10).jpg


In [19]:
df.to_csv('feature_test.csv')